In [7]:
import lancedb
import importlib
from langchain.vectorstores import LanceDB
from langchain.embeddings import OpenAIEmbeddings

# DB specifications
LANCEDB_DIR = "/Users/leon/Documents/study/MA/lancedb"
TABLE_NAME_DOCS = "documents"
TABLE_NAME_CHUNKS = "chunks"

db = lancedb.connect(LANCEDB_DIR)

In [2]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = LanceDB(uri=LANCEDB_DIR, embedding=embeddings, table=db.open_table(TABLE_NAME_CHUNKS))

/var/folders/ll/qhsvkwd97tsbm3310qw2zl2w0000gn/T/ipykernel_30170/2558687755.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [6]:
def retrieve(query: str, k=5):
    retrieved_docs = vector_store.similarity_search(query, k=k)
    return retrieved_docs

In [26]:
from typing import List
from utils import llm

importlib.reload(llm)


def generate(query: str, context: List):
    docs_content = "\n\n".join(doc.page_content for doc in context)
    system_prompt = "You are an expert in question answering from a given context."
    user_prompt = f"Answer the given question as short as possible based on the context, which has been retrieved by similarity search from a knowledge base.\nContext:\n{docs_content}\n\nQuestion: {query}"
    model = "gpt-4.1"

    return llm.call_openai(system_prompt, user_prompt, model, response_format_pydantic=llm.LLMQAResponse)

In [34]:
query = "According to the hospitalization records of Knoxville City Hospital, what is S. Moore's marital status?"

context = retrieve(query)
response = generate(query, context)

print(response.answer)

Widowed


In [32]:
for doc in context:
    print(doc)
    print()

page_content='Hospitalization Record

Basic Information:
Name: S. Moore
Gender: Female
Age: 74
Ethnicity: Caucasian
Marital Status: Married
Occupation: Retired
Address: 60, Newport Street, Knoxville
Admission Time: 11th January
Record Time: 11th January 14:00
Historian: Self
Hospital Name: Knoxville City Hospital

Chief Complaint:
Intermittent wheezing, coughing, and shortness of breath for 2 weeks' metadata={'chunk_id': 1512, 'creation_date': datetime.datetime(2005, 1, 1, 0, 0), 'doc_id': 100209, 'domain': 'Medical', 'outdated_by_chunk_ids': [1725]}

page_content='Hospitalization Record

Basic Information:
Name: S. Moore
Gender: Female
Age: 74
Ethnicity: Caucasian
Marital Status: Widowed
Occupation: Retired
Address: 60, Newport Street, Knoxville
Admission Time: 11th January
Record Time: 11th January 14:00
Historian: Self
Hospital Name: Knoxville City Hospital

Chief Complaint:
Intermittent wheezing, coughing, and shortness of breath for 2 weeks' metadata={'chunk_id': 1230, 'creation_d